In [2]:
import pandas as pd

df = pd.read_csv('../data/raw.csv')

df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53040,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53041,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53042,53040,Is there any way to sleep better? I can't slee...,Anxiety
53043,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [3]:
depression = list()
personality_disorder = list()
bipolar = list()
anxiety = list()
stress = list()
normal = list()
suicide = list()

for i in range(len(df.status)):
    if df.status[i] == "Depression":
        depression.append(df.statement[i])
    elif df.status[i] == "Personality disorder":
        personality_disorder.append(df.statement[i])
    elif df.status[i] == "Bipolar":
        bipolar.append(df.statement[i])
    elif df.status[i] == "Anxiety":
        anxiety.append(df.statement[i])
    elif df.status[i] == "Stress":
        stress.append(df.statement[i])
    elif df.status[i] == "Normal":
        normal.append(df.statement[i])
    elif df.status[i] == "Suicidal":
        suicide.append(df.statement[i])
    else:
        continue

In [4]:
labeled_sentences = list()

for sentence in depression:
    labeled_sentences.append((sentence, "Depression"))

for sentence in personality_disorder:
    labeled_sentences.append((sentence, "Personality Disorder"))

for sentence in bipolar:
    labeled_sentences.append((sentence, "Bipolar"))

for sentence in anxiety:
    labeled_sentences.append((sentence, "Anxiety"))

for sentence in stress:
    labeled_sentences.append((sentence, "Stress"))

for sentence in normal:
    labeled_sentences.append((sentence, "Normal"))

for sentence in suicide:
    labeled_sentences.append((sentence, "Suicidal"))

In [5]:
sentences = [str(sentence) for sentence, label in labeled_sentences]
labels = [str(label) for sentence, label in labeled_sentences]

sentences

['I recently went through a breakup and she said she still wants to be friends so I said I can try doing that but when she talks to me about things it always hurts. I just want to lose feelings so all this pain can stop it hurts so much and I cannot even cry about it. I do not want to hurt her because she said she does not want to never speak to me again but I do not know what to do here. When we were together she always hurt me so I do not know why I still love her. I wish we never met it would be much less painful How do I lose feelings?',
 'I do not know how to navigate these feelings, not that its a new feeling by any stretch. I just do not understand how I can go on from being so pleasantly fine to suicidal and self loathing within in minutes and not be able to step out of it. I have been suffering from poor mental health for over 10 years and I am about to turn 26 next week and I feel so ashamed for how serious my negative thoughts are. Covid was a hard transition especially havi

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', lowercase=True)
tfidf_vector = vectorizer.fit_transform(sentences).toarray()

tfidf_vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
label_arrays = encoder.fit_transform(labels)

label_arrays

array([2, 2, 2, ..., 6, 6, 6], dtype=int64)

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tfidf_vector, label_arrays, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [9]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(x_train.shape[1],)),  # Input layer with TF-IDF features
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer
    tf.keras.layers.Dropout(0.3),  # Dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),  # Another hidden layer
    tf.keras.layers.Dropout(0.3),  # Dropout for regularization
    tf.keras.layers.Dense(len(set(label_arrays)), activation='softmax')  # Output layer for classification
])

c:\Users\Hendra Wijaya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.6009 - loss: 1.0972 - val_accuracy: 0.7555 - val_loss: 0.6409
Epoch 2/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8061 - loss: 0.5354 - val_accuracy: 0.7628 - val_loss: 0.6218
Epoch 3/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8443 - loss: 0.4188 - val_accuracy: 0.7655 - val_loss: 0.6395
Epoch 4/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8824 - loss: 0.3296 - val_accuracy: 0.7660 - val_loss: 0.6782
Epoch 5/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9051 - loss: 0.2633 - val_accuracy: 0.7606 - val_loss: 0.7344
Epoch 6/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9272 - loss: 0.2120 - val_accuracy: 0.7600 - val_loss: 0.8080
Epoch 7/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9430 - loss: 0.1652 - val_accuracy: 0.7549 - val_loss: 0.8717
Epoch 8/10
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9559 - loss: 0.1359 -

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy with Neural Network: {accuracy}")

166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7685 - loss: 1.0896
Test Accuracy with Neural Network: 0.7598039507865906


In [14]:
test = input()
test_vector = vectorizer.transform([test]).toarray()

prediction = model.predict(test_vector)
prediction = encoder.inverse_transform([prediction.argmax()])
print(f"Prediction: {prediction[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: Anxiety


In [17]:
model.save('../models/model.keras')